# Classification attempt using ch7_classification

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} scikit-learn
!conda install --yes --prefix {sys.prefix} xgboost
!conda install --yes --prefix {sys.prefix} seaborn
!conda install --yes --prefix {sys.prefix} matplotlib

In [202]:
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from util import util
from util.VisualizeDataset import VisualizeDataset

In [203]:
dataset = pd.read_csv("./A3_dataset/COMBINED_ch4_result.csv")
print(dataset.columns)
dataset.head()

Index(['Unnamed: 0', 'acc_x', 'acc_y', 'acc_z', 'grav_x', 'grav_y', 'grav_z',
       'lin_x', 'lin_y', 'lin_z', 'gyr_x', 'gyr_y', 'gyr_z',
       'time_since_start', 'timestamp_date', 'timestamp_ms', 'label-jogging',
       'label-cycling', 'label-football', 'label-jumprope', 'label-pushups',
       'label-sitting', 'label-swimming', 'label-tennis', 'label-walking',
       'lin_x_max_freq_ws20', 'lin_y_max_freq_ws20', 'lin_z_max_freq_ws20',
       'lin_x_max_freq_ws40', 'lin_y_max_freq_ws40', 'lin_z_max_freq_ws40'],
      dtype='object')


,Unnamed: 0,acc_x,acc_y,acc_z,grav_x,grav_y,grav_z,lin_x,lin_y,lin_z,...,label-sitting,label-swimming,label-tennis,label-walking,lin_x_max_freq_ws20,lin_y_max_freq_ws20,lin_z_max_freq_ws20,lin_x_max_freq_ws40,lin_y_max_freq_ws40,lin_z_max_freq_ws40
0,0,5.00870,3.5865,6.0478,3.00250,4.03080,8.42070,2.0062,-0.4442,-2.3730,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,9.29790,-9.0417,3.4058,6.11765,3.60885,3.77155,1.2018,2.6037,9.4019,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,13.58710,-21.6699,0.7638,9.23280,3.18690,-0.87760,22.8759,-0.5101,20.9578,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5.62875,0.4956,10.9619,8.47780,-1.96420,-4.52110,4.7023,23.7443,25.2070,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-2.32960,22.6611,21.1600,8.37730,-2.77790,-4.27480,1.3718,25.8676,28.1546,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


# Knn classification

In [204]:
from sklearn.neighbors import KNeighborsClassifier

# Making a single label column
labels = ['label-jogging','label-cycling', 'label-football', 'label-jumprope', 'label-pushups', 'label-sitting', 'label-swimming', 'label-tennis', 'label-walking']
activities = ['jogging','cycling', 'football', 'jumprope', 'pushups', 'sitting', 'swimming', 'tennis', 'walking']

activity_labels = []

for i in range(len(dataset)):
    for j in range(len(labels)):
        if dataset[labels[j]][i] == 1:
            activity_labels.append(activities[j])


In [216]:
from collections import Counter
Counter(activity_labels)

Counter({'jogging': 303,
         'cycling': 6764,
         'football': 9549,
         'jumprope': 2902,
         'pushups': 418,
         'sitting': 17195,
         'swimming': 3034,
         'tennis': 14360,
         'walking': 1435})

In [205]:
# Quickly impute every instance of missing values with median

dataset_filled = dataset.copy()
dataset_columns = dataset_filled.columns
dataset_filled[dataset_columns] = dataset_filled[dataset_columns].fillna(dataset_filled[dataset_columns].mean())

/tmp/ipykernel_20722/3761499697.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataset_filled[dataset_columns] = dataset_filled[dataset_columns].fillna(dataset_filled[dataset_columns].mean())


In [206]:
dataset_filled = dataset_filled.drop(labels + ["Unnamed: 0", "timestamp_date", "timestamp_ms", "time_since_start", 

       # 'lin_x_max_freq_ws20', 'lin_y_max_freq_ws20', 'lin_z_max_freq_ws20', 'lin_x_max_freq_ws40',
       # 'lin_y_max_freq_ws40', 'lin_z_max_freq_ws40'
       
       ], axis=1)

In [207]:
dataset_filled.columns

Index(['acc_x', 'acc_y', 'acc_z', 'grav_x', 'grav_y', 'grav_z', 'lin_x',
       'lin_y', 'lin_z', 'gyr_x', 'gyr_y', 'gyr_z', 'lin_x_max_freq_ws20',
       'lin_y_max_freq_ws20', 'lin_z_max_freq_ws20', 'lin_x_max_freq_ws40',
       'lin_y_max_freq_ws40', 'lin_z_max_freq_ws40'],
      dtype='object')

In [208]:
X_train, X_test, y_train, y_test = train_test_split(dataset_filled, activity_labels, test_size=0.33, shuffle=True)

In [209]:
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)
ypreds=knn_clf.predict(X_test)

In [210]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt

conf_matrix = confusion_matrix(list(ypreds), y_test)
df_cm = pd.DataFrame(conf_matrix, index = activities, columns = activities)

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, cmap="BuPu")
plt.show()

/tmp/ipykernel_20722/1753925427.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize = (10,7))


In [211]:
conf_matrix

array([[1994,   96,   22,   96,   18,    1,   30,   87,   16],
       [  73, 2495,   26,   85,    4,    4,   13,  408,   90],
       [   2,    3,   23,    2,    0,    0,    0,    4,    0],
       [  25,   59,    9,  661,    2,    0,   18,   54,    7],
       [   8,    2,    2,    5,  107,    1,   10,    5,    9],
       [   9,    8,    0,    1,    0, 5687,    8,    0,    0],
       [  15,    8,    6,    4,   16,    0,  894,    9,    3],
       [  80,  414,   13,  105,    4,    0,   13, 4107,   51],
       [   4,   47,    5,   13,    3,    0,    4,   30,  330]])

In [201]:
from sklearn.metrics import f1_score

f1_score(y_test, ypreds, average='weighted')

0.8806866742594254

### Quick test

In [212]:
ypreds_test = []

for i in range(len(y_test)):
    ypreds_test.append("sitting")

f1_score(y_test, ypreds_test, average='weighted')

0.14528443344616512

# Random Forest

In [184]:
from sklearn.ensemble import RandomForestClassifier

clf= RandomForestClassifier()
clf.fit(X_train,y_train)
ypreds=clf.predict(X_test)

In [185]:
conf_matrix = confusion_matrix(list(ypreds), y_test)
df_cm = pd.DataFrame(conf_matrix, index = activities, columns = activities)

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, cmap="BuPu")
plt.show()

/tmp/ipykernel_20722/1978059746.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize = (10,7))


In [186]:
conf_matrix

array([[2042,   17,    9,   74,   11,    5,   19,   28,   10],
       [  33, 2789,   10,   59,    1,    1,    2,  305,   72],
       [   0,    0,   34,    0,    0,    0,    0,    0,    0],
       [  21,    5,    1,  637,    2,    0,    1,   16,    7],
       [   6,    0,    0,    3,  108,    2,    8,    9,    7],
       [   1,    2,    0,    0,    0, 5784,    1,    0,    0],
       [  17,    0,    5,   14,   18,    2,  962,   12,    1],
       [ 106,  302,   19,  128,    2,    2,    9, 4266,   38],
       [   9,   10,    3,   15,    2,    0,    0,   17,  366]])

# Boosting

In [187]:
from xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(X_train,y_train)
ypreds=clf.predict(X_test)

/home/damion/anaconda3/envs/mlqs/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/damion/anaconda3/envs/mlqs/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[13:34:12] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [188]:
conf_matrix = confusion_matrix(list(ypreds), y_test)
df_cm = pd.DataFrame(conf_matrix, index = activities, columns = activities)

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, cmap="BuPu")
plt.show()

/tmp/ipykernel_20722/1978059746.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize = (10,7))


In [189]:
conf_matrix

array([[2045,   19,    1,   63,   12,    5,   20,   31,    8],
       [  34, 2750,    8,   49,    0,    1,    2,  331,   37],
       [   1,    1,   46,    0,    0,    0,    0,    1,    1],
       [  32,   12,    4,  675,    5,    0,    4,   35,   10],
       [   3,    0,    0,    3,  105,    3,    5,    5,    5],
       [   2,    4,    0,    0,    0, 5784,    3,    0,    0],
       [  16,    0,    2,    8,   17,    0,  953,   10,    2],
       [  96,  328,   16,  117,    3,    3,   11, 4228,   36],
       [   6,   11,    4,   15,    2,    0,    4,   12,  402]])